In [1]:
# 3rd party imports
import os
import json
import pandas as pd
from datetime import datetime
import geopandas as gpd
import pyproj
import plotly.express as px
from shapely import wkt

# Configure Notebook
#for plots to be inline
%matplotlib inline 
#for auto_complete 
%config Completer.use_jedi = False 

#hiding warnings
import warnings
warnings.filterwarnings('ignore')

## Timeframe of the storm

The original dataset is too big for the purposes of this example. **We will cut it down to 5 hours, ranging from Feb 17th at 8PM to Feb 18th at 1AM.**
Our map will have a slider covering those 5 hours of plowing data.

In [2]:
#location for each file and folder
start_time = pd.to_datetime('2022-02-17 20:00')
end_time = pd.to_datetime('2022-02-18 01:00')

## Preparing the geometries for neighbourhoods
When plotting with choropleth, you have the option of using the "geometry" column from your original df.
However, this method is very computationally expensive, since a geometry will be loaded for every row in your df (imagine 1000 rows - you'd need a LOT of memory, and time).

An alternative method would be passing a json file with all geometries to be plotted, identified by a common 'id'. Plotly reads the 'id' on the original df, looks up the same 'id' on the json file, and plots accordingly. For our case, this 'id' will be the name of the neighbourhood.

Instead of loading geometries over and over for different rows, this method allows the geometries to be passed only once (from the json), and have their color 'rewritten' over and over, saving memory (which is done by the df).

The city of Toronto's Open Data Portal has geometry data for every neighbourhood in GeoJSON format. It is only a matter of cleaning up the df, and using the neighbourhood name as the 'id' column.

In [3]:
neighbourhoods = gpd.read_file('src/neighbourhoods.geojson')
neighbourhoods.head()

,name_neigh,area,geometry
0,Casa Loma,3.678385e+06,"POLYGON ((-79.41469 43.67391, -79.41485 43.674..."
1,Annex,5.337192e+06,"POLYGON ((-79.39414 43.66872, -79.39588 43.668..."
2,Caledonia-Fairbank,2.955857e+06,"POLYGON ((-79.46021 43.68156, -79.46044 43.681..."
3,Woodbine Corridor,3.052518e+06,"POLYGON ((-79.31485 43.66674, -79.31660 43.666..."
4,Lawrence Park South,6.211341e+06,"POLYGON ((-79.41096 43.70408, -79.41165 43.703..."


In [4]:
#Preparing the geometries for neighbourhoods
neighbourhoods = gpd.read_file('src/neighbourhoods.geojson')

#The index of the json has to be the neighbourhood name
neighbourhoods.index = neighbourhoods['name_neigh']

#Dropping useless columns for this application (it's all about saving memory)
neighbourhoods.drop(['name_neigh', 'area'], axis=1, inplace=True)

#Choropleth mapbox accepts a json for the geometries of neighbourhoods.
neighbourhoods_json = json.loads(neighbourhoods.to_json())
neighbourhoods.head()

,geometry
name_neigh,
Casa Loma,"POLYGON ((-79.41469 43.67391, -79.41485 43.674..."
Annex,"POLYGON ((-79.39414 43.66872, -79.39588 43.668..."
Caledonia-Fairbank,"POLYGON ((-79.46021 43.68156, -79.46044 43.681..."
Woodbine Corridor,"POLYGON ((-79.31485 43.66674, -79.31660 43.666..."
Lawrence Park South,"POLYGON ((-79.41096 43.70408, -79.41165 43.703..."


# The Plow data
In order to be available on github, the plow dataframe had to be saved into 6 different chunks, which can me concatenated before starting out analysis. The plow df has already been cleaned, and shows a few columns:

**completed_time**: All of the data has been grouped by hour. This column is originally NOT in datetime format

**neighbourhood**: The neighbourhood of interest per row

**route_name**: The plow data is filtered by type of road, including Local roads, Collectors, Arterials, Expressways etc.

**routetype**: The district where the neighbourhood sits in. Toronto/East York, Scarborough, Etobicoke and North York.

**length**: The length of road that has been plowed, per neighbourhood, and per type of road.

**total_length**: using the openn data portal, the total length of each type of roada was calculated per neighbourhood.

**length_ratio**: the percentage of type of road plowed per neighbourhood (100*length/total_length)

**geometry**: the geometry shape for that neighbourhood.

In [5]:
list=[]
for file in os.listdir('src'):
    if 'df_' in file:
        print(file)
        df = pd.read_csv(os.path.join(os.getcwd(), r'src', file))
        list.append(df)
df = pd.concat(list, axis=0, ignore_index=True)
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'])
gdf.head()

df_5.csv
df_4.csv
df_3.csv
df_2.csv
df_0.csv
df_1.csv


,completed_time,neighbourhood,route_name,routetype,length,total_length,length_ratio,geometry
0,2022-02-18T00:00:00,Humewood-Cedarvale,Expressway Ramp,0,0.0,0.0,0.0,"POLYGON ((-79.41849 43.68363, -79.41913 43.683..."
1,2022-02-18T00:00:00,Humewood-Cedarvale,Laneway,0,0.0,0.0,0.0,"POLYGON ((-79.41849 43.68363, -79.41913 43.683..."
2,2022-02-18T00:00:00,Humewood-Cedarvale,Other,0,0.0,0.0,0.0,"POLYGON ((-79.41849 43.68363, -79.41913 43.683..."
3,2022-02-18T00:00:00,Humewood-Cedarvale,Access Road,0,0.0,0.0,0.0,"POLYGON ((-79.41849 43.68363, -79.41913 43.683..."
4,2022-02-18T00:00:00,Humewood-Cedarvale,Collector Ramp,0,0.0,0.0,0.0,"POLYGON ((-79.41849 43.68363, -79.41913 43.683..."


### Picking only one type of road
For this example, we'll investigate the Major Arterial roads, and how they were plowed by the city.

In [6]:
level = 'Major Arterial' #out of ['Local', 'Minor Arterial', 'Laneway', 'Major Arterial', 'Other','Collector', 'Major Arterial Ramp', 'Expressway', 'Expressway Ramp','Pending', 'Collector Ramp', 'Access Road']

In [7]:
gdf = gdf[gdf.route_name == level]

#We need to work with datetime for the labels
gdf['completed_time'] = pd.to_datetime(gdf['completed_time'])

#Also for labels, we'll rename the ratio column
gdf.rename(columns={'length_ratio':'% plowed'}, inplace=True)

#Plotly does not automatically sort for time, so we'll make sure it is sorted beforehand.
gdf.sort_values(by='completed_time', inplace=True)

#for the slider labels, we'll use datetime's .strftime() function.
gdf['Day and Hour'] = gdf['completed_time'].dt.strftime("%d at %Hh")

#To save memory, we'll drop unnecessary columns
gdf.drop(['completed_time','route_name','routetype','length', 'total_length', 'geometry'], axis=1, inplace=True)
gdf.head()

,neighbourhood,% plowed,Day and Hour
7689,Rexdale-Kipling,0.0,17 at 20h
7953,Hillcrest Village,0.0,17 at 20h
7965,Ionview,0.0,17 at 20h
7977,Kennedy Park,0.0,17 at 20h
7989,L'Amoreaux,0.0,17 at 20h


## Summing up what we have at this point
We have the name of the neighbourhood, the plowed %, and the day and hour, in string format.
We also have a json with geometry information for every neighbourhood.

**Both of these files will be linked by a common identification column.** We are using the neighbourhood name as the common identificator. 

**The JSON file has the name of neighbourhood as it's index** (it has to be the index), and the **df has a column named 'neighbourhood'**, with the names of neighbourhoods. This is how plotly connects one to the other.

Choropleth has the argument 'locations', which is where we'll pass df['neighbourhoods']

We are ready to start potting the map.

In [ ]:
#using plotly for an animated choropleth map
fig = px.choropleth_mapbox(data_frame=gdf,
                           geojson=neighbourhoods_json,
                           locations=gdf.neighbourhood,
                           color='% plowed',
                           center={'lat':43.72, 'lon':-79.38},
                           mapbox_style='open-street-map',
                           zoom=9,
                           color_continuous_scale='blues',
                           range_color=(0, 50),
                           animation_frame='Day and Hour',
                           width=800,
                           height=600)
fig.write_html('plow_map.html')
fig.show()